Import necessary libraries

In [ ]:
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
from sklearn.metrics         import accuracy_score

Connect your Google Drive. 
Since the files are quite big, it is much more convinient to upload them in a cloud service.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Read prepared train-test files

In [ ]:
link_to_drive = '/content/drive/MyDrive/DL_Final_Project/'

In [ ]:
X_val   = np.load(link_to_drive + 'X_val.npy')
X_train = np.load(link_to_drive + 'X_train.npy')
y_val   = np.load(link_to_drive + 'y_val.npy')
y_train = np.load(link_to_drive + 'y_train.npy')
X_test  = np.load(link_to_drive + 'X_test.npy')
y_test  = pd.read_csv(link_to_drive + 'Test.csv')

Build a Convolutional Neural Network (CNN).

In [ ]:
def model_NN(X_train):
  model = Sequential()
  model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Dropout(rate=0.25))
  model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(BatchNormalization())
  model.add(Dropout(rate=0.25))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(rate=0.5))
  model.add(Dense(43, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

Train the CNN model

In [ ]:
epochs = 15
batch_size = 64
model = model_NN(X_train)
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))

Plot the graphs for accuracy and loss of both datasets

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy and Loss')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

Test the trained model with test dataset

In [ ]:
predict_x=model.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)

labels = y_test['ClassId'].values
print(accuracy_score(labels, classes_x))